In [1]:
import config
import cx_Oracle
import re
import pandas as pd

pattern = re.compile(r'\s+')

In [2]:
# connect to Oracle DB
connection = None
try:
    connection = cx_Oracle.connect(
        config.username,
        config.password,
        config.dsn,
        encoding = "UTF-8"
    )
    print("Connected")
    print(connection.version)
except cx_Oracle.Error as error:
    print(error)

cursor = connection.cursor()

Connected
12.2.0.1.0


In [9]:
# set up style view and get the geometrytype of that style's feature
view_name = "V_HV_TRF_SYM_1"
feature_name = view_name[2:]
sql = "Select g3e_geometrytype from G3E_COMPONENT where G3E_NAME = '{name}'".format(name = feature_name)
cursor.execute(sql)
geometrytype = cursor.fetchall()[0][0]

In [11]:
# hardcode condition to know which styletable for the given style view
if geometrytype == "PolygonGeometry":
    styletable = 'G3E_AREASTYLE'
elif geometrytype == "PolylineGeometry":
    styletable = 'G3E_LINESTYLE'
elif geometrytype == "OrientedPointGeometry":
    styletable = 'G3E_POINTSTYLE'
elif geometrytype == "TextPointGeometry":
    styletable = 'G3E_TEXTSTYLE'
elif geometrytype == "CompositePolylineGeometry":
    styletable = 'G3E_COMPOSITELINESTYLE'

In [12]:
# extract text that create the given style view
sql = "SELECT TEXT FROM USER_VIEWS WHERE VIEW_NAME = '{view_name}'".format(view_name = view_name)
cursor.execute(sql)
fetch = cursor.fetchall()
txts = fetch[0][0]
txts = txts.rpartition("CASE")[2]
txts = txts.rpartition("ELSE")[0]
txts = re.sub(pattern,'',txts)
txts = txts.split("WHEN")

print(txts)

['', "TRIM(D.CODETEXT)IN('PPA','PPM','PPR','PPX')ANDTRIM(E.CODETEXT)='RECTIFIER'THEN14800010", "TRIM(D.CODETEXT)IN('PPA','PPM','PPR','PPX')THEN14800020", "TRIM(D.CODETEXT)IN('ABA','ABM','ABR','ABX')ANDTRIM(E.CODETEXT)='RECTIFIER'THEN14800030", "TRIM(D.CODETEXT)IN('ABA','ABM','ABR','ABX')THEN14800040", "TRIM(D.CODETEXT)='INS'ANDTRIM(E.CODETEXT)='RECTIFIER'THEN14800050", "TRIM(D.CODETEXT)='INS'THEN14800060", "TRIM(D.CODETEXT)IN('RES','ABN')ANDTRIM(E.CODETEXT)='RECTIFIER'THEN14800070", "TRIM(D.CODETEXT)IN('RES','ABN')THEN14800080", "TRIM(D.CODETEXT)='APA'ANDTRIM(E.CODETEXT)='RECTIFIER'THEN14800090", "TRIM(D.CODETEXT)='APA'THEN14800100"]


In [13]:
# extract cause(logic condition) and effect(styleid) into dict
logics = []

spl_chr = "THEN"
for txt in txts:
    if txt != "":
        logic = {}
        logic[txt.rpartition(spl_chr)[2]] = txt.rpartition(spl_chr)[0]
        logics.append(logic)
    
print(logics)

[{'14800010': "TRIM(D.CODETEXT)IN('PPA','PPM','PPR','PPX')ANDTRIM(E.CODETEXT)='RECTIFIER'"}, {'14800020': "TRIM(D.CODETEXT)IN('PPA','PPM','PPR','PPX')"}, {'14800030': "TRIM(D.CODETEXT)IN('ABA','ABM','ABR','ABX')ANDTRIM(E.CODETEXT)='RECTIFIER'"}, {'14800040': "TRIM(D.CODETEXT)IN('ABA','ABM','ABR','ABX')"}, {'14800050': "TRIM(D.CODETEXT)='INS'ANDTRIM(E.CODETEXT)='RECTIFIER'"}, {'14800060': "TRIM(D.CODETEXT)='INS'"}, {'14800070': "TRIM(D.CODETEXT)IN('RES','ABN')ANDTRIM(E.CODETEXT)='RECTIFIER'"}, {'14800080': "TRIM(D.CODETEXT)IN('RES','ABN')"}, {'14800090': "TRIM(D.CODETEXT)='APA'ANDTRIM(E.CODETEXT)='RECTIFIER'"}, {'14800100': "TRIM(D.CODETEXT)='APA'"}]


In [21]:
check = list(logics[0].keys())[0]
print(check)
sqlstyle = "SELECT * FROM {table} WHERE g3e_sno = '{styleid}'".format(table = styletable, styleid = check)
print(sqlstyle)
try:
    df=pd.read_sql(sqlstyle,con=connection)
    data = df.to_dict(orient = 'list')
    print(data)
except Exception as e:
        print(e)      


14800010
SELECT * FROM G3E_POINTSTYLE WHERE g3e_sno = '14800010'
Execution failed on sql: SELECT * FROM G3E_POINTSTYLE WHERE g3e_sno = '14800010'
ORA-03114: not connected to ORACLE
unable to rollback


In [20]:
# get the relevant information for the given style id
styles = []
for logic in logics:
    style = {}
    styleid = list(logic.keys())[0]
    sqlstyle = "SELECT * FROM {table} WHERE g3e_sno = '{styleid}'".format(table = styletable, styleid = styleid)
    df=pd.read_sql(sqlstyle,con=connection)
    data = df.to_dict(orient = 'list')
    style["styleid"] = data
    styles.append(style)

DatabaseError: Execution failed on sql: SELECT * FROM G3E_POINTSTYLE WHERE g3e_sno = '14800010'
ORA-03135: connection lost contact
Process ID: 5500
Session ID: 252 Serial number: 55218
unable to rollback

In [5]:
# close connection
if connection:
    cursor.close()
    connection.close()
    print("Connection closed")

Connection closed
